# Extract attributes HLS

In [20]:
from finn.util.visualization import showInNetron

##############
partitioned = False # no GenericPartition nodes
#file_path = "/workspace/results/syn_v1/end2end_quartznet_syn_v1_hlssynth_annotated.onnx"
syn_id = "syn_v4"
file_path = "/workspace/results/"+syn_id+"/end2end_quartznet_syn_v4_hlssynth_annotated.onnx"
##############

showInNetron(file_path)

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_v4/end2end_quartznet_syn_v4_hlssynth_annotated.onnx' at http://0.0.0.0:8081


In [23]:
######
# Change model attribute of generic partition (because the path to partitioned models is different
# from the docker container and from the singularity on the qce-alveo server)
######

#from finn.core.modelwrapper import ModelWrapper
#from finn.custom_op.registry import getCustomOp

#model = ModelWrapper("/workspace/results/syn_v0/end2end_quartznet_folded.onnx")

#for n in model.graph.node:
#    if n.op_type=="GenericPartition":
#        inst = getCustomOp(n)
#        path = inst.get_nodeattr("model")
#        new_path = path.replace("/finn_dev_mirza", "/syn_v0")
#        inst.set_nodeattr("model", new_path)
    
#model.save("/workspace/results/syn_v0/end2end_quartznet_folded_renamed.onnx")    


# Create FINN estimates (AnnotateCycles, AnnotateResources)

In [21]:
from finn.core.modelwrapper import ModelWrapper
from finn.transformation.fpgadataflow.annotate_cycles import AnnotateCycles
from finn.transformation.fpgadataflow.annotate_resources import AnnotateResources
from finn.custom_op.registry import getCustomOp

#model = ModelWrapper("/workspace/results/syn_v0/end2end_quartznet_folded_renamed.onnx")
model = ModelWrapper(file_path)

if partitioned:
    for n in model.graph.node:
        if n.op_type=="GenericPartition":
            inst = getCustomOp(n)
            model_path = inst.get_nodeattr("model")
            model_partition = ModelWrapper(model_path)
            model_partition = model_partition.transform(AnnotateCycles())
            model_partition = model_partition.transform(AnnotateResources('estimate'))
            model_partition.save(model_path)
else:
    model = model.transform(AnnotateCycles())
    model = model.transform(AnnotateResources('estimate'))
    model.save(file_path)   
            
#showInNetron("/workspace/results/syn_v0/end2end_quartznet_folded_renamed.onnx")

In [22]:
from finn.util.visualization import showInNetron
showInNetron(file_path)

Stopping http://0.0.0.0:8081
Serving '/workspace/results/syn_v4/end2end_quartznet_syn_v4_hlssynth_annotated.onnx' at http://0.0.0.0:8081


# Write relevant node attributes to .csv file

In [23]:
## TODO:
# make template for cropped model

from finn.core.modelwrapper import ModelWrapper
from finn.util.basic import get_by_name
from finn.custom_op.registry import getCustomOp
from finn.util.fpgadataflow import is_fpgadataflow_node
import numpy as np
import ast

#################
partition_num = 0
#################

time_format = "us" # print time target in 'us' or 'ns'
clk_target = 10 # clk target set for synthesis (see test script)

#model = ModelWrapper("/workspace/results/syn_v0/partitioning_repartition/partition_"+str(partition_num)+".onnx")
model = ModelWrapper(file_path)

output_dict = {}
for n in model.graph.node:
    print(n.name)
    if is_fpgadataflow_node(n):
        inst = getCustomOp(n)
        if n.op_type=="FMPadding_Batch":
            res_type = '-'
            ram_style = '-'
            mem_mode = '-'
        elif n.op_type=="ConvolutionInputGenerator1D":
            res_type = '-'
            ram_style = inst.get_nodeattr("ram_style")
            mem_mode = '-'
        elif n.op_type=="Vector_Vector_Activate_Batch":
            res_type = inst.get_nodeattr("resType")
            ram_style = '-'
            mem_mode = '-'
        elif n.op_type=="StreamingFCLayer_Batch":
            res_type = inst.get_nodeattr("resType")
            ram_style = inst.get_nodeattr("ram_style")
            mem_mode = inst.get_nodeattr("mem_mode")
        elif n.op_type=="Thresholding_Batch":
            res_type = '-'
            ram_style = inst.get_nodeattr("ram_style")
            mem_mode = inst.get_nodeattr("mem_mode")            
        elif n.op_type=="DuplicateStreams_Batch":
            res_type = '-'
            ram_style = '-'
            mem_mode = '-'
        elif n.op_type=="AddStreams_Batch":
            res_type = '-'
            ram_style = '-'
            mem_mode = '-'
        else:
            print("Op_type not recognized!")
            break
        
        #res_dict = get_by_name(n.attribute, "res_hls").s.decode('utf-8')
        res_dict = inst.get_nodeattr("res_hls")
        res_estimate = get_by_name(n.attribute, "res_estimate").s.decode('utf-8')
        try:
            pe = get_by_name(n.attribute, "PE").i
        except AttributeError:
            pe=None
        try:
            simd = get_by_name(n.attribute, "SIMD").i
        except AttributeError:
            simd=None
        #est_clk = get_by_name(n.attribute, "est_clk_hls").s.decode('utf-8')
        est_clk = inst.get_nodeattr("est_clk_hls")
        #est_latency_hls = get_by_name(n.attribute, "est_latency_hls").s.decode('utf-8')
        est_latency_hls = inst.get_nodeattr("est_latency_hls")
        finn_cycles = get_by_name(n.attribute, "cycles_estimate").i
        node_name_list = n.name.split("_")
        p_id = node_name_list[1]
        node_name = n.name.replace(node_name_list[0]+"_"+node_name_list[1]+"_","")
        
        if not partitioned:
            node_name = p_id+"_"+node_name #make node names unique

        output_dict[node_name] = {}
        output_dict[node_name]["name"] = {"partition_id": p_id, "node_name": node_name}
        output_dict[node_name]["folding"] = {"PE": pe, "SIMD": simd}
        output_dict[node_name]["res_types"] = {"res_type": res_type, "ram_style": ram_style, "mem_mode": mem_mode}
        output_dict[node_name]["timing_hls"] = {"CLK_target": clk_target,
                                             "CLK_est": est_clk,
                                             "Cycles (best-ave-worst)": est_latency_hls}
        output_dict[node_name]["res_hls"] = res_dict
        output_dict[node_name]["FINN cycles"] = {"exp_cycles": finn_cycles, "rtlsim_cycles": "-"}
        output_dict[node_name]["res_estimate"] = res_estimate

print(output_dict)

if partitioned:
    f = open("info_layer_{}.csv".format(partition_num), "a")
else:
    f = open("info_layer_{}.csv".format(syn_id), "a")
#f = open("test_info_layer_{}.txt".format(p_id), "a")
for node_name in output_dict.keys():
    for col in output_dict[node_name].keys():
        if col=="name":
            partition_id = output_dict[node_name][col]["partition_id"]
            node_name = output_dict[node_name][col]["node_name"]
            f.write(str(partition_id)+","+str(node_name))
        if col=='folding':
            pe = output_dict[node_name][col]['PE']
            simd = output_dict[node_name][col]['SIMD']
            f.write(","+str(pe)+","+str(simd))
        if col=="res_types":
            res_type = output_dict[node_name][col]["res_type"]
            ram_style = output_dict[node_name][col]["ram_style"]
            mem_mode = output_dict[node_name][col]["mem_mode"]
            f.write(","+str(res_type)+","+str(ram_style)+","+str(mem_mode))
        if col=='timing_hls':
            clk_target = output_dict[node_name][col]["CLK_target"]
            clk_est_raw = output_dict[node_name][col]["CLK_est"]
            if clk_est_raw!="-": #which is the default, meaning that HLS not ran for this layer
                assert(clk_est_raw[-2:]=="ns"),"clk_estimation is not in ns!"
                clk_est = clk_est_raw.replace(" ns","")
            else:
                clk_est = clk_est_raw
            f.write(","+str(clk_target)+","+str(clk_est))
            cycles_raw = output_dict[node_name][col]["Cycles (best-ave-worst)"]
            if cycles_raw!="-":
                cycles = cycles_raw.split(',')
                cycles_list=[]
                for substring in cycles:
                    cycles_list.append(int(''.join(c for c in substring if c.isdigit())))
                for c in cycles_list:
                    f.write(","+str(c))
                if time_format == "ns":
                    worst_time = round(cycles_list[-1]*float(clk_est)*1E-9,3)
                elif time_format == "us":
                    worst_time = round(cycles_list[-1]*float(clk_est)*1E-6,3)
                f.write(","+str(worst_time)) # worst time
            else:
                f.write(","+cycles_raw+","+cycles_raw+","+cycles_raw) #latency
                f.write(","+"-") # worst time
        if col=='res_hls':
            if output_dict[node_name][col]=="-":
                f.write(","+"-"+","+"-"+","+"-"+","+"-"+","+"-"+","+"-"+","+"-"+","+"-"+","+"-"+","+"-")
            else:
                hls_dict = ast.literal_eval(output_dict[node_name][col])
                bram_18k = hls_dict["BRAM_18K"]
                bram_18k_fraction = round(int(bram_18k)/4032*100,3)
                ff = hls_dict["FF"]
                ff_fraction = round(int(ff)/2607360*100,3)
                lut = hls_dict["LUT"]
                lut_fraction = round(int(lut)/1303680*100,3)
                dsp48e = hls_dict["DSP48E"]
                dsp48e_fraction = round(int(dsp48e)/9024*100,3)
                uram = hls_dict["URAM"]
                uram_fraction = round(int(uram)/960*100,3)
                f.write(","+str(bram_18k)+","+str(bram_18k_fraction)+"%")
                f.write(","+str(ff)+","+str(ff_fraction)+"%")
                f.write(","+str(lut)+","+str(lut_fraction)+"%")
                f.write(","+str(dsp48e)+","+str(dsp48e_fraction)+"%")
                f.write(","+str(uram)+","+str(uram_fraction)+"%")
        if col=="FINN cycles":
            exp_cycles = output_dict[node_name][col]["exp_cycles"]
            rtlsim_cycles = output_dict[node_name][col]["rtlsim_cycles"]
            f.write(","+str(exp_cycles)+","+str(rtlsim_cycles))
        if col=="res_estimate":
            res_finn_dict = ast.literal_eval(output_dict[node_name][col])
            bram_18k = res_finn_dict["BRAM_18K"]
            bram_efficiency = round(res_finn_dict["BRAM_efficiency"],3)
            lut = res_finn_dict["LUT"]
            uram = res_finn_dict["URAM"]
            uram_efficiency = round(res_finn_dict["URAM_efficiency"],3)
            dsp = res_finn_dict["DSP"]
            f.write(","+str(bram_18k)+","+str(bram_efficiency)+","+str(lut)+","+str(uram) \
                    +","+str(uram_efficiency)+","+str(dsp))
    f.write("\n")
f.close()
    


    

pt_0_Transpose_0
pt_0_FMPadding_Batch_0
pt_0_ConvolutionInputGenerator1D_0
pt_0_Vector_Vector_Activate_Batch_0
pt_1_StreamingFCLayer_Batch_0
pt_1_DuplicateStreams_Batch_0
pt_1_FMPadding_Batch_0
pt_1_StreamingFCLayer_Batch_1
pt_1_ConvolutionInputGenerator1D_0
pt_1_Vector_Vector_Activate_Batch_0
pt_1_StreamingFCLayer_Batch_2
pt_1_FMPadding_Batch_4
pt_1_ConvolutionInputGenerator1D_4
pt_1_Vector_Vector_Activate_Batch_4
pt_1_StreamingFCLayer_Batch_6
pt_1_AddStreams_Batch_0
pt_4_Thresholding_Batch_0
pt_4_DuplicateStreams_Batch_0
pt_4_StreamingFCLayer_Batch_0
pt_4_FMPadding_Batch_4
pt_4_ConvolutionInputGenerator1D_4
pt_4_Vector_Vector_Activate_Batch_4
pt_4_StreamingFCLayer_Batch_5
pt_4_AddStreams_Batch_0
pt_7_Thresholding_Batch_0
pt_7_DuplicateStreams_Batch_0
pt_7_FMPadding_Batch_0
pt_7_StreamingFCLayer_Batch_0
pt_7_ConvolutionInputGenerator1D_0
pt_7_Vector_Vector_Activate_Batch_0
pt_7_StreamingFCLayer_Batch_1
pt_7_FMPadding_Batch_4
pt_7_ConvolutionInputGenerator1D_4
pt_7_Vector_Vector_Activa